# Property Scraper - Data Analysis

This notebook analyzes scraped property listing data that's automatically committed to the repository by GitHub Actions.

**Features:**
- Loads CSV data from the `data/` directory
- Provides data analysis and visualizations
- Works in both local Jupyter and Google Colab
- No GitHub CLI setup needed!

---

## 1. Setup and Installation

In [ ]:
import sys
import os
from pathlib import Path

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

print(f"Running in: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")

# Install required packages
if IN_COLAB:
    print("Installing packages...")
    !pip install -q pandas matplotlib seaborn
else:
    print("Packages should already be installed locally")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Packages imported successfully")

## 2. Clone Repository (Colab Only)

If you're in Colab, clone the repository to access the data:

In [ ]:
if IN_COLAB:
    REPO_URL = "https://github.com/yourusername/scraping_listings.git"  # Change this!
    
    # Clone the repository
    !git clone {REPO_URL}
    
    # Change to repo directory
    os.chdir("scraping_listings")
    print(f"✓ Repository cloned, working directory: {os.getcwd()}")
else:
    print(f"Working directory: {os.getcwd()}")

## 3. Load Data from Repository

In [ ]:
# Data directory
DATA_DIR = Path("data")

if not DATA_DIR.exists():
    print(f"❌ Data directory not found: {DATA_DIR.absolute()}")
    print("Make sure you're in the correct directory!")
else:
    print(f"✓ Data directory found: {DATA_DIR.absolute()}")

In [ ]:
def find_all_csv_files():
    """Find all CSV files in data directory."""
    csv_files = list(DATA_DIR.glob("*.csv"))
    
    # Organize by type
    pages_files = [f for f in csv_files if "pages" in f.name]
    details_files = [f for f in csv_files if "details" in f.name]
    
    return {
        'all': csv_files,
        'pages': pages_files,
        'details': details_files
    }

csv_files = find_all_csv_files()

print(f"Found CSV files:")
print(f"  Total: {len(csv_files['all'])}")
print(f"  Pages: {len(csv_files['pages'])}")
print(f"  Details: {len(csv_files['details'])}")

if csv_files['all']:
    print(f"\nSample files:")
    for f in list(csv_files['all'])[:5]:
        print(f"  - {f.name}")
else:
    print("\n⚠️ No CSV files found. The scrapers haven't run yet or the repo is empty.")

## 4. Load and Merge Data

In [ ]:
def load_and_merge_csvs(file_list, scraper_name=None):
    """Load and merge multiple CSV files into a single DataFrame."""
    if scraper_name:
        file_list = [f for f in file_list if scraper_name in f.name]
    
    if not file_list:
        print(f"No files found for scraper: {scraper_name}")
        return None
    
    print(f"Loading {len(file_list)} files...")
    
    dfs = []
    for f in file_list:
        try:
            df = pd.read_csv(f)
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {f.name}: {e}")
    
    if not dfs:
        return None
    
    # Merge all DataFrames
    merged_df = pd.concat(dfs, ignore_index=True)
    
    # Remove duplicates based on URL
    if 'url' in merged_df.columns:
        before = len(merged_df)
        merged_df = merged_df.drop_duplicates(subset=['url'], keep='last')
        after = len(merged_df)
        if before != after:
            print(f"  Removed {before - after} duplicates")
    
    # Convert date_time to datetime
    if 'date_time' in merged_df.columns:
        merged_df['date_time'] = pd.to_datetime(merged_df['date_time'])
    
    print(f"✓ Loaded {len(merged_df)} records")
    return merged_df

# Load data for each scraper
scrapers = ['fincaraiz', 'metrocuadrado', 'mercado-libre']
data = {}

for scraper in scrapers:
    print(f"\n--- Loading {scraper} data ---")
    
    pages_df = load_and_merge_csvs(csv_files['pages'], scraper)
    details_df = load_and_merge_csvs(csv_files['details'], scraper)
    
    data[scraper] = {
        'pages': pages_df,
        'details': details_df
    }

print("\n" + "="*60)
print("Data loading complete!")

## 5. Data Overview

In [ ]:
def show_data_summary():
    """Display summary of all loaded data."""
    summary_data = []
    
    for scraper, dfs in data.items():
        pages_count = len(dfs['pages']) if dfs['pages'] is not None else 0
        details_count = len(dfs['details']) if dfs['details'] is not None else 0
        
        summary_data.append({
            'Scraper': scraper.title(),
            'Pages Records': pages_count,
            'Detail Records': details_count,
            'Total': pages_count + details_count
        })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
    
    # Total
    total_records = summary_df['Total'].sum()
    print(f"\nTotal records across all scrapers: {total_records:,}")

show_data_summary()

## 6. Detailed Analysis

### Choose a scraper to analyze:

In [ ]:
# Select scraper to analyze
SELECTED_SCRAPER = 'fincaraiz'  # Change to 'metrocuadrado' or 'mercado-libre'

df_details = data[SELECTED_SCRAPER]['details']

if df_details is None or len(df_details) == 0:
    print(f"No data available for {SELECTED_SCRAPER}")
else:
    print(f"Analyzing {SELECTED_SCRAPER} data...")
    print(f"Total records: {len(df_details):,}")
    print(f"\nColumns: {list(df_details.columns)}")
    print(f"\nFirst few records:")
    display(df_details.head())

In [ ]:
# Parse the 'information' JSON column
if df_details is not None and 'information' in df_details.columns:
    print("Parsing information column...")
    
    # Parse JSON strings
    import ast
    
    def safe_parse(x):
        try:
            if isinstance(x, str):
                return ast.literal_eval(x)
            return x
        except:
            return {}
    
    df_details['info_parsed'] = df_details['information'].apply(safe_parse)
    
    # Extract common fields
    if len(df_details) > 0:
        sample_info = df_details['info_parsed'].iloc[0]
        print(f"\nAvailable fields in information:")
        for key in sample_info.keys():
            print(f"  - {key}")
        
        # Extract some fields as columns
        for field in ['title', 'pricing', 'price', 'location', 'bedrooms', 'bathrooms', 'area']:
            if field in sample_info:
                df_details[field] = df_details['info_parsed'].apply(
                    lambda x: x.get(field, None) if isinstance(x, dict) else None
                )
        
        print("\n✓ Information parsed")

### Basic Statistics

In [ ]:
if df_details is not None and len(df_details) > 0:
    print("Data Statistics:\n")
    
    # Show scraped dates
    if 'date_time' in df_details.columns:
        print(f"Date range: {df_details['date_time'].min()} to {df_details['date_time'].max()}")
        print(f"Scraping sessions: {df_details['date_time'].dt.date.nunique()}")
    
    # Show location distribution
    if 'location' in df_details.columns:
        print(f"\nTop 10 Locations:")
        location_counts = df_details['location'].value_counts().head(10)
        display(location_counts)
    
    # Show price statistics
    if 'pricing' in df_details.columns or 'price' in df_details.columns:
        price_col = 'pricing' if 'pricing' in df_details.columns else 'price'
        print(f"\nPrice Statistics:")
        
        # Extract numeric prices
        df_details['price_numeric'] = df_details[price_col].astype(str).str.extract(r'([\d,.]+)')[0]
        df_details['price_numeric'] = df_details['price_numeric'].str.replace(',', '').str.replace('.', '')
        df_details['price_numeric'] = pd.to_numeric(df_details['price_numeric'], errors='coerce')
        
        valid_prices = df_details['price_numeric'].dropna()
        if len(valid_prices) > 0:
            print(f"  Count: {len(valid_prices):,}")
            print(f"  Mean: ${valid_prices.mean():,.0f}")
            print(f"  Median: ${valid_prices.median():,.0f}")
            print(f"  Min: ${valid_prices.min():,.0f}")
            print(f"  Max: ${valid_prices.max():,.0f}")

### Visualizations

In [ ]:
if df_details is not None and len(df_details) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{SELECTED_SCRAPER.title()} - Data Analysis', fontsize=16)
    
    # 1. Scraping timeline
    if 'date_time' in df_details.columns:
        df_details['date'] = df_details['date_time'].dt.date
        daily_counts = df_details['date'].value_counts().sort_index()
        axes[0, 0].plot(daily_counts.index, daily_counts.values, marker='o')
        axes[0, 0].set_title('Records Scraped Over Time')
        axes[0, 0].set_xlabel('Date')
        axes[0, 0].set_ylabel('Number of Records')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Price distribution
    if 'price_numeric' in df_details.columns:
        valid_prices = df_details['price_numeric'].dropna()
        if len(valid_prices) > 0:
            # Remove outliers for better visualization
            q1, q3 = valid_prices.quantile([0.25, 0.75])
            iqr = q3 - q1
            filtered_prices = valid_prices[(valid_prices >= q1 - 1.5*iqr) & (valid_prices <= q3 + 1.5*iqr)]
            
            axes[0, 1].hist(filtered_prices, bins=30, edgecolor='black')
            axes[0, 1].set_title('Price Distribution (outliers removed)')
            axes[0, 1].set_xlabel('Price')
            axes[0, 1].set_ylabel('Frequency')
    
    # 3. Top locations
    if 'location' in df_details.columns:
        top_locations = df_details['location'].value_counts().head(10)
        axes[1, 0].barh(range(len(top_locations)), top_locations.values)
        axes[1, 0].set_yticks(range(len(top_locations)))
        axes[1, 0].set_yticklabels(top_locations.index)
        axes[1, 0].set_title('Top 10 Locations')
        axes[1, 0].set_xlabel('Number of Listings')
    
    # 4. Bedrooms distribution
    if 'bedrooms' in df_details.columns:
        bedroom_counts = df_details['bedrooms'].value_counts().sort_index().head(10)
        axes[1, 1].bar(bedroom_counts.index.astype(str), bedroom_counts.values)
        axes[1, 1].set_title('Bedrooms Distribution')
        axes[1, 1].set_xlabel('Number of Bedrooms')
        axes[1, 1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

## 7. Export Merged Data

In [ ]:
# Export merged data to CSV
output_dir = Path("merged_data")
output_dir.mkdir(exist_ok=True)

for scraper, dfs in data.items():
    if dfs['details'] is not None and len(dfs['details']) > 0:
        output_file = output_dir / f"{scraper}_merged_details.csv"
        dfs['details'].to_csv(output_file, index=False)
        print(f"✓ Exported {scraper} data to {output_file}")
        print(f"  Records: {len(dfs['details']):,}")

print(f"\nAll merged data saved to: {output_dir.absolute()}")

## 8. Download Merged Data (Colab Only)

In [ ]:
if IN_COLAB:
    from google.colab import files
    import zipfile
    
    # Create ZIP of merged data
    zip_filename = "merged_property_data.zip"
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for csv_file in output_dir.glob("*.csv"):
            zipf.write(csv_file, csv_file.name)
    
    print(f"Downloading {zip_filename}...")
    files.download(zip_filename)

## 9. Custom Analysis

Use this cell for your own custom analysis:

In [ ]:
# Your custom analysis here
# Example: Filter by location
if df_details is not None:
    # Example filter
    # filtered_df = df_details[df_details['location'].str.contains('Chapinero', na=False)]
    # display(filtered_df[['url', 'title', 'pricing', 'location']].head(10))
    pass